In [1]:
import librosa
import numpy as np
import IPython.display as ipd
import os
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

## Constants

In [2]:
quantity = 64
n_mfcc = 20
n_threads = 32
other_path = r'audiobooks\data\ru_open_stt\private_buriy_audiobooks_2\0'
start_path = r'augmented'

## Preparing data

#### Preparing start samples

In [3]:
%%time
def kernel(files):
    tmp = []
    for file in files:
        x, sr = librosa.load(file)
        mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=len(x)//quantity, n_mfcc=n_mfcc)
        tmp.append(mfcc)
    return tmp


files = list(os.scandir(start_path))
files_per_thread = len(files) // n_threads 
grouped_files = []
tmp = []
i = 0
for file in files:
    tmp.append(file)
    i += 1
    if i >= files_per_thread:
        grouped_files.append(tmp)
        tmp = []
        i = 0
if i != 0:
    grouped_files.append(tmp)

pool = ThreadPool(n_threads)
X_start = pool.map(kernel, grouped_files)
pool.close()
pool.join()

X_start = [j for i in np.arange(len(X_start)) for j in X_start[i]]
X_start = np.array(X_start).reshape(*np.shape(X_start), 1)
y_start = np.ones(np.shape(X_start)[0])
np.shape(X_start)

Wall time: 2min 8s


(1542, 20, 65, 1)

#### Preparing other samples

In [24]:
%%time
def kernel(folder):
    tmp = []
    with os.scandir(folder) as entries:
        for entry in entries:
            if entry.is_file():
                x, sr = librosa.load(entry)
                mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=len(x)//quantity, n_mfcc=n_mfcc)
                tmp.append(mfcc)
    return tmp


folders = list(os.scandir(other_path))
pool = ThreadPool(len(folders))
X_other = pool.map(kernel, folders)
pool.close()
pool.join()

X_other = [j for i in np.arange(len(X_other)) for j in X_other[i]]
X_other = np.array(X_other).reshape(*np.shape(X_other), 1)
y_other = np.zeros(np.shape(X_other)[0])
np.shape(X_other)

Wall time: 1min 35s


(3359, 20, 65, 1)

## Loading data

#### Loading start samples

In [24]:
data = np.load('data/start.npz')
X_start = data['sample']
y_start = data['label']
X_start.shape

(426, 20, 65, 1)

#### Loading other samples

In [4]:
data = np.load('data/other.npz')
X_other = data['sample']
y_other = data['label']
X_other.shape

(3359, 20, 65, 1)

## Merging

In [5]:
X = np.append(X_start, X_other, axis=0)
Y = np.append(y_start, y_other, axis=0)
X.shape

(4901, 20, 65, 1)

## Saving samples

#### Saving start samples

In [6]:
np.savez_compressed('data/start', sample=X_start, label=y_start)

#### Saving other samples

In [21]:
np.savez_compressed('data/other', sample=X_other, label=y_other)

#### Saving merged samples

In [7]:
np.savez_compressed('data/merged', sample=X, label=Y)